In [ ]:
import sys
sys.path.append('../input/autoaugment/')
# sys.path.append('../input/model-pkl/')
from autoaugment import ImageNetPolicy

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets,  transforms, models
from collections import OrderedDict
from torch.optim.lr_scheduler import *
from collections import OrderedDict
!pip install adabound
!pip install efficientnet_pytorch
import adabound
import numpy as np
import time


data_dir = '../input/cat-dog-data5/cat_dog_data/Cat_Dog_data'

train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       ImageNetPolicy(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
                                                           
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=16, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, num_workers=16, pin_memory=True)

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b3')
model._fc = nn.Linear(1536, 2)
model.load_state_dict(torch.load('../input/model-pkl/efficientnet-b3.pkl'))

criterion = nn.CrossEntropyLoss()
# optimizer = adabound.AdaBound(model.parameters(), lr=1e-6, final_lr=0.1)
optimizer = optim.RMSprop(model.parameters(), lr=1e-7, momentum=0.9, eps=1e-3, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size=int(2.4 * len(trainloader)), gamma=0.97)
device = torch.device('cuda: 0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

def do_deep_learning(model, trainloader, testloader, epochs, print_every, criterion, optimizer, device=device):
    epochs = epochs
    print_every = print_every
    steps = 0
    
    model.to(device)    
    
    for e in range(epochs):
        start_time = time.time()
        running_loss = 0
        for ii, (inputs, labels) in enumerate(trainloader):
            steps += 1
            scheduler.step()
            
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                model.eval()
                accuracy = 0
                count = 0
                with torch.no_grad():
                    for i, data in enumerate(testloader):
                        images, labels = data
                        images, labels = images.to(device), labels.to(device)
                        outputs = model(images)
                        _, predicted = torch.max(outputs.data, 1)
                        count += labels.size(0)
                        accuracy += (predicted == labels).sum().item()
                        if i == 0:
                            break
                            
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every),
                      "Time: {:.2f}s".format(time.time()-start_time),
                     "Accuracy: {:.5f}%".format(100 * accuracy / count))
                      
                start_time = time.time()
                running_loss = 0
                model.train()
        
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %.5f %%' % (100 * correct / total))
    
if __name__ == "__main__":
    
    do_deep_learning(model, trainloader, testloader, 1, 100, criterion, optimizer, device)
    
    torch.save(model.state_dict(),'efficientnet-b3.pkl')